In [3]:
import pandas as pd
from influxdb import DataFrameClient

In [4]:
user = 'root'
password = 'root'
dbname = 'meta22'
host='localhost'
port=34000
bsize = 5000 #batch size
# Temporarily avoid line protocol time conversion issues #412, #426, #431.
protocol = 'json'
client = DataFrameClient(host, port, user, password, dbname)

In [5]:
client.create_database(dbname)
client.query("show databases")

ResultSet({'(u'databases', None)': [{u'name': u'meta22'}]})

In [6]:
df=pd.read_csv('data/temperature_ft.csv')

In [8]:
meta=pd.read_excel('data/metadata.xlsx')
meta.columns=[i if i!='Point ID' else 'monitor_id' for i in meta.columns]

In [9]:
for measure in ['gas','water','electricity','heating','temperature']:
    df=pd.read_csv('data/'+measure+'_ft.csv').drop('measurement_unit',axis=1)
    merged=pd.merge(meta,df,on=['monitor_id']).set_index('ts')
    merged.index = pd.to_datetime(merged.index)
    client.write_points(merged, measure, protocol=protocol, 
                        tag_columns=list(meta.columns), 
                        field_columns=['measurement'], batch_size=bsize)
    print(measure)

gas
water
electricity
heating
temperature


In [10]:
meta

,monitor_id,Original Point label:,Company Name,Site Name,Location 1,Location 2,Location 3,Postcode,Building type,Building name,Building year,Bedrooms,Measurement,Measurement type,Measurement unit,Point label
0,217,A to H water / electric (217),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,NaN,NaN,2008,NaN,Calculation - Ratio,NaN,%,NaN
1,89,Academic Bld 30min (89),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,Academic Building,NaN,2008,NaN,Electricity,Volume,kWh,Academic Building
2,98,Block J Elec 30min (98),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,Student Hall,Block J,2008,91.0,Electricity,Volume,kWh,Block J
3,95,Mach Plant MC Elec 30min (95),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,NaN,NaN,2008,NaN,Electricity,Volume,kWh,Mechnical Plant MC
4,86,Main Incomer Elec 30min (86),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,NaN,NaN,2008,NaN,Electricity,Volume,kWh,Main Incomer
5,94,Pavilion Elec 30min (94),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,NaN,Pavilion,2008,NaN,Electricity,Volume,kWh,Pavilion
6,96,Student Hall1 Elec 30min (96),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,Student Hall,Block A,2008,47.0,Electricity,Volume,kWh,Block A
7,97,Student Hall2 Elec 30min (97),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,Student Hall,Block B,2008,47.0,Electricity,Volume,kWh,Block B
8,87,Student Hall3 30min (87),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,Student Hall,Block C,2008,61.0,Electricity,Volume,kWh,Block C
9,88,Student Hall4 Elec 30min (88),UPP,Woolf College,University of Kent,Canterbury,Kent,CT2 7BQ,Student Hall,Block D,2008,47.0,Electricity,Volume,kWh,Block D


In [ ]:
print("Delete database: " + dbname)
client.drop_database(dbname)